### Dolphin Staph Isolate Contaminant Split
### Author: Shawn Higdon
### Date: January 10, 2019

> This analysis is for the analysis of a bacterial isolate recovered from the blow hole of a Dolphin specimen under investigation by the Weimer Lab at UC Davis. The isolate in question was previously determined to be a co-culture of E. coli and S. aureus. The goal for this analysis will be to determine taxa present within the isolate, followed by spliting of fastq reads that comprise the genomes within each species comprising the isolate.

#### Two workflows will be attempted to separate the genomes within the co-cultured genome sequence:

1. Read Splitting using BBsplit and associated reference genomes
2. Fastq read splitting following metagenome assembly and subsequent Genome binning with Metabat

**Pre-Analysis Step**

1. Quality trim reads using trimmomatic

**Workflow 1 Analysis Steps**

1. Generate minhash sketch using __Sourmash__
2. Classify minhash sketch using __Sourmash__
3. __Download__ relevant reference genomes form __NCBI__
4. Isolate fastq reads of organism 1 by maping fastq reads to reference genome 1 using __BBSplit__
5. Isolate fastq reads of organism 2 by maping fastq reads to reference genome 2 using __BBSplit__
6. Assembly individual genomes using __Megahit__
7. Generate Minhash Sketch of separated microbial genomes using __Sourmash__
8. Classify separated microbial genomes using __Sourmash__
9. Compute assembly statistics using __Quast__ and __BBtools__

**Workflow 2 Analysis Steps**

1. Assemble metagenome using __Megahit__
2. Map reads to assembly using __BWA__
3. Bin metagenome using __Metabat__
4. Generate split fastq reads for each bin from original raw reads using __Pysam__
5. Assemble split reads using __Megahit__
6. Generate Minhash Sketches for subsequent split read assemblies using __Megahit__
7. Classify Minhash Sketches for assemblies using __Sourmash__
8. Compute assembly statistics using __Quast__ and __BBtools__


> Sourmash version 2.0.0a11


In [1]:
module load bio

Module bio/1.0 loaded 
>

#### Pre-Analysis

> Quality Trim the reads

In [10]:
pwd && ls

/group/weimergrp/smh/Dolphin_staph
BCW_7425_R1.fastq.gz  BCW_7425_R2.fastq.gz  Untitled.ipynb


In [16]:
for R1 in *R1.fastq.gz; do echo $R1; R2=`echo $R1 | sed 's/_R1/_R2/'`; bname=`echo $R1 | sed 's/_R1.\+//'`; bname=`basename $bname`; echo $R2; echo "bname is $bname"; trimmomatic PE -threads 24 $R1 $R2 ${bname}.R1.trim.fq ${bname}.orphan_R1.fq ${bname}.R2.trim.fq ${bname}.orphan_R2.fq ILLUMINACLIP:TruSeq3-PE-2.fa:2:40:15:8:TRUE LEADING:2 TRAILING:2 SLIDINGWINDOW:4:15 MINLEN:50; done

BCW_7425_R1.fastq.gz
BCW_7425_R2.fastq.gz
bname is BCW_7425
TrimmomaticPE: Started with arguments:
 -threads 24 BCW_7425_R1.fastq.gz BCW_7425_R2.fastq.gz BCW_7425.R1.trim.fq BCW_7425.orphan_R1.fq BCW_7425.R2.trim.fq BCW_7425.orphan_R2.fq ILLUMINACLIP:TruSeq3-PE-2.fa:2:40:15:8:TRUE LEADING:2 TRAILING:2 SLIDINGWINDOW:4:15 MINLEN:50
Using PrefixPair: 'TACACTCTTTCCCTACACGACGCTCTTCCGATCT' and 'GTGACTGGAGTTCAGACGTGTGCTCTTCCGATCT'
Using Long Clipping Sequence: 'AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGTA'
Using Long Clipping Sequence: 'AGATCGGAAGAGCACACGTCTGAACTCCAGTCAC'
Using Long Clipping Sequence: 'GTGACTGGAGTTCAGACGTGTGCTCTTCCGATCT'
Using Long Clipping Sequence: 'TACACTCTTTCCCTACACGACGCTCTTCCGATCT'
ILLUMINACLIP: Using 1 prefix pairs, 4 forward/reverse sequences, 0 forward only sequences, 0 reverse only sequences
Quality encoding detected as phred33
Input Read Pairs: 1076753 Both Surviving: 1057182 (98.18%) Forward Only Surviving: 14569 (1.35%) Reverse Only Surviving: 2389 (0.22%) Dropped: 2613 (0.

In [17]:
ls

BCW_7425.orphan_R1.fq  BCW_7425.R1.trim.fq   TruSeq3-PE-2.fa
BCW_7425.orphan_R2.fq  BCW_7425_R2.fastq.gz  Untitled.ipynb
BCW_7425_R1.fastq.gz   BCW_7425.R2.trim.fq


### Workflow 2 

> Beginning analysis with Workflow 2

#### MEGAHIT Assembly

In [22]:
megahit -1 ${bname}.R1.trim.fq -2 ${bname}.R2.trim.fq -o workflow2/megahit -t 24

62.0Gb memory in total.
Using: 56.592Gb.
MEGAHIT v1.1.1
--- [Thu Jan 10 14:39:38 2019] Start assembly. Number of CPU threads 24 ---
--- [Thu Jan 10 14:39:38 2019] Available memory: 67516657664, used: 60764991897
--- [Thu Jan 10 14:39:38 2019] Converting reads to binaries ---
    [read_lib_functions-inl.h  : 209]     Lib 0 (BCW_7425.R1.trim.fq,BCW_7425.R2.trim.fq): pe, 2114364 reads, 151 max length
    [utils.h                   : 126]     Real: 3.8486	user: 3.3635	sys: 0.4799	maxrss: 164808
--- [Thu Jan 10 14:39:43 2019] k list: 21,29,39,59,79,99,119,141 ---
--- [Thu Jan 10 14:39:43 2019] Extracting solid (k+1)-mers for k = 21 ---
--- [Thu Jan 10 14:39:54 2019] Building graph for k = 21 ---
--- [Thu Jan 10 14:39:57 2019] Assembling contigs from SdBG for k = 21 ---
--- [Thu Jan 10 14:40:00 2019] Local assembling k = 21 ---
--- [Thu Jan 10 14:40:06 2019] Extracting iterative edges from k = 21 to 29 ---
--- [Thu Jan 10 14:40:11 2019] Building graph for k = 29 ---
--- [Thu Jan 10 14:40:13 

In [28]:
ls workflow2/megahit

done  final.contigs.fa  intermediate_contigs  log  opts.txt


#### Read Mapping with BWA

> BWA

In [29]:
module load bwa

Module bwa/0.7.17.r1188 loaded
>

In [31]:
metaASM=workflow2/megahit/final.contigs.fa # set variable for metagneome assembly
WF2_MAP_FOLDER=workflow2/mapping
mkdir -p $WF2_MAP_FOLDER
ls workflow2

mapping  megahit


In [32]:
bwa index $metaASM # index the assembly contigs

[bwa_index] Pack FASTA... 0.10 sec
[bwa_index] Construct BWT for the packed sequence...
[bwa_index] 3.51 seconds elapse.
[bwa_index] Update BWT... 0.06 sec
[bwa_index] Pack forward-only FASTA... 0.05 sec
[bwa_index] Construct SA from BWT and Occ... 1.16 sec
[main] Version: 0.7.17-r1188
[main] CMD: bwa index workflow2/megahit/final.contigs.fa
[main] Real time: 5.182 sec; CPU: 4.880 sec


In [36]:
# map fastq reads used to make assembly to the assembly index
bwa mem -t 24 $metaASM BCW_7425.R1.trim.fq BCW_7425.R2.trim.fq > $WF2_MAP_FOLDER/BCW_7425.aln.sam

[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 1632202 sequences (240000214 bp)...
[M::process] read 482162 sequences (70960251 bp)...
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (365, 776303, 504, 395)
[M::mem_pestat] analyzing insert size distribution for orientation FF...
[M::mem_pestat] (25, 50, 75) percentile: (117, 187, 271)
[M::mem_pestat] low and high boundaries for computing mean and std.dev: (1, 579)
[M::mem_pestat] mean and std.dev: (196.44, 105.12)
[M::mem_pestat] low and high boundaries for proper pairs: (1, 733)
[M::mem_pestat] analyzing insert size distribution for orientation FR...
[M::mem_pestat] (25, 50, 75) percentile: (231, 292, 360)
[M::mem_pestat] low and high boundaries for computing mean and std.dev: (1, 618)
[M::mem_pestat] mean and std.dev: (296.73, 98.71)
[M::mem_pestat] low and high boundaries for proper pairs: (1, 747)
[M::mem_pestat] analyzing insert size distribution for orientation RF...
[M::mem_pestat] (25, 50, 75) p

In [37]:
ls workflow2/mapping

BCW_7425.aln.sam


#### Convert SAM to BAM

> Samtools

In [38]:
module load samtools

Module samtools/1.9 loaded 
>

In [40]:
samtools faidx $metaASM #index assembly for samtools

In [42]:
samtools import $metaASM workflow2/mapping/BCW_7425.aln.sam $WF2_MAP_FOLDER/BCW_7425.aln.bam

In [47]:
samtools sort $WF2_MAP_FOLDER/BCW_7425.aln.bam -o $WF2_MAP_FOLDER/BCW_7425.aln.bam.sorted

In [49]:
samtools index $WF2_MAP_FOLDER/BCW_7425.aln.bam.sorted

In [50]:
ls workflow2/mapping

BCW_7425.aln.bam         BCW_7425.aln.bam.sorted.bai
BCW_7425.aln.bam.sorted  BCW_7425.aln.sam


#### Binning

> Metabat

In [51]:
module load metabat

Module metabat 2017-01-17 Loaded.
>

In [53]:
mkdir -p workflow2/metabat
WF2_METABAT_FOLDER=workflow2/metabat

In [54]:
# calculate read coverage, coverage variance & tetranucleotide frequencies for each contig
jgi_summarize_bam_contig_depths --outputDepth $WF2_METABAT_FOLDER/BCW_7425_depth_var.txt $WF2_MAP_FOLDER/*.sorted

Output depth matrix to workflow2/metabat/BCW_7425_depth_var.txt
Output matrix to workflow2/metabat/BCW_7425_depth_var.txt
Opening bam: workflow2/mapping/BCW_7425.aln.bam.sorted
Consolidating headers
Processing bam files
Thread 0 processing: BCW_7425.aln.bam.sorted
Thread 0 finished: BCW_7425.aln.bam.sorted with 2138951 reads and 2110356 readsWellMapped
Creating depth matrix file: workflow2/metabat/BCW_7425_depth_var.txt
Closing most bam files
Closing last bam file
Finished


In [55]:
# run MetaBAT
metabat -i $metaASM -a $WF2_METABAT_FOLDER/BCW_7425_depth_var.txt --verysensitive -o $WF2_METABAT_FOLDER/BCW_7425 -v > $WF2_METABAT_FOLDER/BCW_7425-metabat-log.txt

[Info] Correlation binning won't be applied since the number of samples (1) < minSamples (10)


In [56]:
ls workflow2/metabat

BCW_7425.1.fa  BCW_7425.2.fa  BCW_7425_depth_var.txt  BCW_7425-metabat-log.txt


In [57]:
cat workflow2/metabat/BCW_7425-metabat-log.txt

[00:00:00] Using p1 90.0%, p2 85.0%, p3 90.0%, minProb 75.0%, minBinned 30%, minCV 1.0, minContig 2500, minContigByCorr 1000, minCorr 90%, paired 0, and 0 bootstrapping
[00:00:00] Finished reading 241 contigs. Number of target contigs >= 2500 are 97, and [1000 and 2500) are 28 
[00:00:00] Finished reading 241 contigs (using 125 including 28 short contigs) and 1 coverages from workflow2/metabat/BCW_7425_depth_var.txt
[00:00:00] Finished TNF calculation.                                  
[00:00:00] Finished building a probabilistic graph. (125 vertices and 1030 edges)          
[00:00:00] 1st round binning 100.0% (125 of 125), ETA 0:00:00
[00:00:00] Leftover contigs before fish_more: 44.00% (55 out of 125)
[00:00:00] Leftover contigs after fish_more_by_friends_membership (roughly): 40.00% (50 out of 125), 2 bins   
[00:00:00] Leftover contigs after fish_more (roughly): 40.00% (50 out of 125)
[00:00:00] Leftover contigs after fish_more_by_corr (roughly): 40.00% (50 out of 125)
[00:00:00] 

#### Split fastq reads for each bin

> Pysam

> Use perl one-liner to extract contig IDs from each metabat bin

In [103]:
perl -ne 'm/^>(\S+)/ and print("$1\t1\n")' < workflow2/metabat/BCW_7425.1.fa >> workflow2/BCW_7425_bin_map.txt

In [104]:
perl -ne 'm/^>(\S+)/ and print("$1\t2\n")' < workflow2/metabat/BCW_7425.2.fa >> workflow2/BCW_7425_bin_map.txt

> Show tsv map file

In [105]:
cat workflow2/BCW_7425_bin_map.txt

k141_238	1
k141_237	1
k141_211	1
k141_250	1
k141_151	1
k141_93	1
k141_296	1
k141_109	1
k141_242	1
k141_144	1
k141_227	1
k141_139	1
k141_294	1
k141_150	1
k141_248	1
k141_281	1
k141_283	1
k141_265	1
k141_223	1
k141_254	1
k141_262	1
k141_271	1
k141_86	1
k141_287	1
k141_191	1
k141_233	1
k141_277	1
k141_131	1
k141_257	1
k141_166	1
k141_189	1
k141_208	1
k141_210	1
k141_89	1
k141_290	1
k141_295	1
k141_113	1
k141_221	1
k141_114	1
k141_82	1
k141_187	1
k141_284	1
k141_78	2
k141_192	2
k141_214	2
k141_234	2
k141_244	2
k141_133	2
k141_81	2
k141_286	2
k141_252	2
k141_246	2
k141_96	2
k141_219	2
k141_168	2
k141_259	2
k141_102	2
k141_175	2
k141_152	2
k141_106	2
k141_247	2
k141_193	2
k141_293	2
k141_188	2
k141_84	2
k141_226	2
k141_180	2
k141_125	2
k141_179	2
k141_253	2
k141_159	2
k141_80	2
k141_183	2
k141_182	2
k141_278	2


In [106]:
module unload bio
source activate pysam

(pysam) 

: 1

In [113]:
cat bin-bam-boom.py # view script

#!/usr/bin/env python
#
# python bin-bam-boom.py pooled-reads.bam contig-to-bin-map.tsv outprefix

import pysam
import sys

inbam = sys.argv[1]
mapfile = sys.argv[2]
prefix = sys.argv[3]

# Load contig-->bin map into a dictionary
# Create an output file for each bin
outfiles = dict()
contig_bin_map = dict()
for line in open(mapfile, 'r'):
    contigid, binid = line.strip().split()
    contigid = contigid.split(' ')[0]
    contig_bin_map[contigid] = binid
    if binid not in outfiles:
        outfilename = prefix + '.' + binid + '.fastq'
        outfiles[binid] = open(outfilename, 'w')
outfilename = prefix + '.unbinned.fastq'
outfiles['unbinned'] = open(outfilename, 'w')

# Iterate over all alignments
bam = pysam.AlignmentFile(inbam, 'rb')
for alignment in bam:
    if alignment.is_secondary or alignment.is_supplementary:
        continue # Ignore these ones

    contigid = bam.get_reference_name(alignment.tid)
    if contigid  in contig_bin_map:
         binid = contig_bin_map[contigid]

: 1

In [108]:
python bin-bam-boom.py workflow2/mapping/BCW_7425.aln.bam workflow2/BCW_7425_bin_map.txt workflow2/BCW_7425 # run script

(pysam) 

: 1

In [111]:
ls -lrth workflow2

total 699M
drwxrwsr-x 3 smhigdon weimergrp 4.0K Jan 10 15:26 megahit
drwxrwsr-x 2 smhigdon weimergrp 4.0K Jan 10 15:58 mapping
drwxrwsr-x 2 smhigdon weimergrp 4.0K Jan 10 16:54 metabat
-rw-rw-r-- 1 smhigdon weimergrp  816 Jan 10 16:57 BCW_7425_bin_map.txt
-rw-rw-r-- 1 smhigdon weimergrp 325M Jan 10 17:02 BCW_7425.1.fastq
-rw-rw-r-- 1 smhigdon weimergrp  61M Jan 10 17:02 BCW_7425.unbinned.fastq
-rw-rw-r-- 1 smhigdon weimergrp 315M Jan 10 17:02 BCW_7425.2.fastq
(pysam) 

: 1

#### Classify split fastq

> Sourmash

In [112]:
head workflow2/BCW_7425.1.fastq

@M02034:368:000000000-B942D:1:1101:17676:1923/1
GCAAACCAATAACAGCAAAGATGTCGCGATTAAAGGGCAGGGCTTTTTCCAGGTGATGTTGCCAGATGGCTCATCAGCCTATACCCGTGACGGCTCTTTCCAGGTGGATCAGAACGGGCAGCTGGTGACGGCTGGTGGTTTTCAGGTGCA
+
1/GF?11G?1GF1F11GGDF?/>//</>F21HHGFB/0G?<>/0011000FE@1@B0>10B11FBB0>@121B00FB122B/>///A/AB/AB00011000FAB1AAD21FAFA0AA11B001BA0AE0A111EEAEF@1111FC>>11A
@M02034:368:000000000-B942D:1:1101:17676:1923/2
CGCCAGCCGGTGGCACAGTCTTCCGTACAAACCACCTTACCATCCGGATTACAAATCGGTAC
+
1>>A111>11111A0A00BFFEFG010A011AAFEFFG1101DAB///AA1AA11AF//A/A
@M02034:368:000000000-B942D:1:1101:14879:1956/1
ATAAAAGAACTGACAACCTTTTATTATCCCTCACGTAAAACGATGGCATCAAAA
(pysam) 

: 1

> Split reads are interleaved

In [114]:
source deactivate

In [116]:
module load bio

Module bio/1.0 loaded 
>

In [118]:
# megahit --12 workflow2/BCW_7425.1.fastq -t 24 -o workflow2/megahit/BCW_7425.1

# there are an odd number of total reads that were extracted using the pysam method.

# This implies that there are orphan reads within the subset.

# These will be removed using the khmer script 'extract-paired-reads.py'

62.0Gb memory in total.
Using: 56.592Gb.
MEGAHIT v1.1.1
--- [Fri Jan 11 10:00:24 2019] Start assembly. Number of CPU threads 24 ---
--- [Fri Jan 11 10:00:24 2019] Available memory: 67516657664, used: 60764991897
--- [Fri Jan 11 10:00:24 2019] Converting reads to binaries ---
    [ERROR] [read_lib_functions-inl.h : 203]: PE library number of reads is odd: 986067!
    [ERROR] [read_lib_functions-inl.h : 204]: File(s): workflow2/BCW_7425.1.fastq
Error occurs when running "megahit_asm_core buildlib"; please refer to workflow2/megahit/BCW_7425.1/log for detail
[Exit code 1]


: 1

#### Extract Paired Reads

> BCW_7425.1

> Khmer v2.0

In [120]:
extract-paired-reads.py -d workflow2 -p workflow2/BCW_7425.1_paired.fastq -s workflow2/BCW_7425.1_orphans.fastq workflow2/BCW_7425.1.fastq


|| This is the script extract-paired-reads.py in khmer.
|| You are running khmer version 2.0
|| You are also using screed version 0.9
||
|| If you use this script in a publication, please cite EACH of the following:
||
||   * MR Crusoe et al., 2015. http://dx.doi.org/10.12688/f1000research.6924.1
||
|| Please see http://khmer.readthedocs.org/en/latest/citations.html for details.

reading file "workflow2/BCW_7425.1.fastq"
outputting interleaved pairs to "BCW_7425.1_paired.fastq"
outputting orphans to "BCW_7425.1_orphans.fastq"
... 400000
... 500000
... 600000
... 700000
... 800000
... 900000
DONE; read 986067 sequences, 487296 pairs and 11475 singletons
wrote to: BCW_7425.1_paired.fastq and BCW_7425.1_orphans.fastq


#### Assemble BCW_7425.1 

> Megahit

In [127]:
megahit --12 workflow2/BCW_7425.1_paired.fastq -t 24 -o workflow2/megahit/BCW_7425.1

62.0Gb memory in total.
Using: 56.592Gb.
MEGAHIT v1.1.1
--- [Fri Jan 11 10:21:19 2019] Start assembly. Number of CPU threads 24 ---
--- [Fri Jan 11 10:21:19 2019] Available memory: 67516657664, used: 60764991897
--- [Fri Jan 11 10:21:19 2019] Converting reads to binaries ---
    [read_lib_functions-inl.h  : 209]     Lib 0 (workflow2/BCW_7425.1_paired.fastq): interleaved, 974592 reads, 151 max length
    [utils.h                   : 126]     Real: 3.0701	user: 1.8496	sys: 0.2537	maxrss: 84312
--- [Fri Jan 11 10:21:22 2019] k list: 21,29,39,59,79,99,119,141 ---
--- [Fri Jan 11 10:21:22 2019] Extracting solid (k+1)-mers for k = 21 ---
--- [Fri Jan 11 10:21:27 2019] Building graph for k = 21 ---
--- [Fri Jan 11 10:21:29 2019] Assembling contigs from SdBG for k = 21 ---
--- [Fri Jan 11 10:21:31 2019] Local assembling k = 21 ---
--- [Fri Jan 11 10:21:33 2019] Extracting iterative edges from k = 21 to 29 ---
--- [Fri Jan 11 10:21:36 2019] Building graph for k = 29 ---
--- [Fri Jan 11 10:21:37

#### Assemble BCW_7425.2

> Megahit

In [119]:
megahit --12 workflow2/BCW_7425.2.fastq -t 24 -o workflow2/megahit/BCW_7425.2

62.0Gb memory in total.
Using: 56.592Gb.
MEGAHIT v1.1.1
--- [Fri Jan 11 10:06:29 2019] Start assembly. Number of CPU threads 24 ---
--- [Fri Jan 11 10:06:29 2019] Available memory: 67516657664, used: 60764991897
--- [Fri Jan 11 10:06:29 2019] Converting reads to binaries ---
    [read_lib_functions-inl.h  : 209]     Lib 0 (workflow2/BCW_7425.2.fastq): interleaved, 944526 reads, 151 max length
    [utils.h                   : 126]     Real: 1.6803	user: 1.4422	sys: 0.2357	maxrss: 84036
--- [Fri Jan 11 10:06:31 2019] k list: 21,29,39,59,79,99,119,141 ---
--- [Fri Jan 11 10:06:31 2019] Extracting solid (k+1)-mers for k = 21 ---
--- [Fri Jan 11 10:06:35 2019] Building graph for k = 21 ---
--- [Fri Jan 11 10:06:37 2019] Assembling contigs from SdBG for k = 21 ---
--- [Fri Jan 11 10:06:39 2019] Local assembling k = 21 ---
--- [Fri Jan 11 10:06:42 2019] Extracting iterative edges from k = 21 to 29 ---
--- [Fri Jan 11 10:06:44 2019] Building graph for k = 29 ---
--- [Fri Jan 11 10:06:45 2019] 

#### Compute Minhash Signatures

> Sourmash v.2.0.0a11

In [128]:
ls workflow2/megahit/BCW_7425.1

done  final.contigs.fa  intermediate_contigs  log  opts.txt


In [129]:
mkdir -p workflow2/sourmash

In [131]:
sourmash compute -h

usage: sourmash [-h] [--protein] [--no-protein] [--dna] [--no-dna] [-q]
                [--input-is-protein] [-k KSIZES] [-n NUM_HASHES]
                [--check-sequence] [-f] [-o OUTPUT] [--singleton]
                [--merge MERGED] [--name-from-first] [--track-abundance]
                [--scaled SCALED] [--seed SEED] [--randomize]
                [--license LICENSE]
                filenames [filenames ...]

positional arguments:
  filenames             file(s) of sequences

optional arguments:
  -h, --help            show this help message and exit
  --protein             build protein signatures (default: False)
  --no-protein          do not build protein signatures
  --dna                 build DNA signatures (default: True)
  --no-dna              do not build DNA signatures
  -q, --quiet           suppress non-error output
  --input-is-protein    Consume protein sequences - no translation needed.
  -k KSIZES, --ksizes KSIZES
                        comma-separated list of k-

##### BCW_7425.1

In [175]:
sourmash compute -k 31 --scaled 2000 -o workflow2/sourmash/BCW_7425.1-k31.sig workflow2/megahit/BCW_7425.1/final.contigs.fa

setting num_hashes to 0 because --scaled is set
computing signatures for files: workflow2/megahit/BCW_7425.1/final.contigs.fa
Computing signature for ksizes: [31]
Computing only nucleotide (and not protein) signatures.
Computing a total of 1 signature(s).
... reading sequences from workflow2/megahit/BCW_7425.1/final.contigs.fa
calculated 1 signatures for 46 sequences in workflow2/megahit/BCW_7425.1/final.contigs.fa
saved 1 signature(s). Note: signature license is CC0.


##### BCW_7425.2

In [176]:
sourmash compute -k 31 --scaled 2000 -o workflow2/sourmash/BCW_7425.2-k31.sig workflow2/megahit/BCW_7425.2/final.contigs.fa

setting num_hashes to 0 because --scaled is set
computing signatures for files: workflow2/megahit/BCW_7425.2/final.contigs.fa
Computing signature for ksizes: [31]
Computing only nucleotide (and not protein) signatures.
Computing a total of 1 signature(s).
... reading sequences from workflow2/megahit/BCW_7425.2/final.contigs.fa
calculated 1 signatures for 42 sequences in workflow2/megahit/BCW_7425.2/final.contigs.fa
saved 1 signature(s). Note: signature license is CC0.


#### Classify Signatures

> Sourmash v.2.0.0a11

In [159]:
module unload bio

In [170]:
sourmash lca


sourmash lca <command> [<args>] - work with taxonomic information.

** Commands can be:

index <taxonomy.csv> <output_db name> <signature [...]>  - create LCA database
classify --db <db_name [...]> --query <signature [...]>  - classify genomes
gather <signature> <db_name [...]>                       - classify metagenomes
summarize --db <db_name [...]> --query <signature [...]> - summarize mixture
rankinfo <db_name [...]>                                 - database rank info
compare_csv <csv1> <csv2>                                - compare spreadsheets

** Use '-h' to get subcommand-specific help, e.g.

sourmash lca index -h



: 1

In [3]:
sourmash lca classify --db /home/smhigdon/mnt/genbank-k31.lca.json.gz --query workflow2/sourmash/BCW_7425.1-k31.sig

loaded 1 LCA databases. ksize=31, scaled=10000k-k31.lca.json.gz
finding query signatures...
outputting classifications to stdout
ID,status,superkingdom,phylum,class,order,family,genus,species,strain
workflow2/megahit/BCW_7425.1/final.contigs.fa,found,Bacteria,Proteobacteria,Gammaproteobacteria,Enterobacterales,Enterobacteriaceae,Escherichia,Escherichia coli,
classified 1 signatures total


> Minhash signature for __BCW_7425.1__ was successfully classified to the species level as __*E. coli*__

In [13]:
sourmash lca classify --db /home/smhigdon/mnt/genbank-k31.lca.json.gz --query workflow2/sourmash/BCW_7425.2-k31.sig

loaded 1 LCA databases. ksize=31, scaled=10000k-k31.lca.json.gz
finding query signatures...
outputting classifications to stdout
ID,status,superkingdom,phylum,class,order,family,genus,species,strain
workflow2/megahit/BCW_7425.2/final.contigs.fa,found,Bacteria,Firmicutes,Bacilli,Bacillales,Staphylococcaceae,Staphylococcus,,
classified 1 signatures total


> Minhash signature for __BCW_7425.2__ was successfully classified to the genus level as __Staphylococcus__

In [4]:
module load bio

Module bio/1.0 loaded 
>

#### BCW_7425.1 Assembly Stats

> QUAST

In [6]:
quast


QUAST: QUality ASsessment Tool for Genome Assemblies
Version: 4.1, build 26.05.2016 14:25 

Usage: python /share/apps/bio/bio/bin/quast [options] <files_with_contigs>

Options:
-o  --output-dir  <dirname>   Directory to store all result files [default: quast_results/results_<datetime>]
-R                <filename>  Reference genome file
-G  --genes       <filename>  File with gene coordinates in the reference
-m  --min-contig  <int>       Lower threshold for contig length [default: 500]
-t  --threads     <int>       Maximum number of threads [default: 25% of CPUs]

These are basic options. To see the full list, use --help



In [7]:
mkdir -p workflow2/quast && ls workflow2

BCW_7425.1.fastq          BCW_7425.2.fastq         mapping  quast
BCW_7425.1_orphans.fastq  BCW_7425_bin_map.txt     megahit  sourmash
BCW_7425.1_paired.fastq   BCW_7425.unbinned.fastq  metabat  test


In [8]:
quast -o workflow2/quast/BCW_7425.1 -t 24 workflow2/megahit/BCW_7425.1/final.contigs.fa

/share/apps/bio/bio/opt/quast-4.1/quast.py -o workflow2/quast/BCW_7425.1 -t 24 workflow2/megahit/BCW_7425.1/final.contigs.fa

Version: 4.1, build 26.05.2016 14:25

System information:
  OS: Linux-4.15.0-39-generic-x86_64-with-debian-buster-sid (linux_64)
  Python version: 2.7.15
  CPUs number: 24

Started: 2019-01-11 15:01:14

Logging to /group/weimergrp/smh/Dolphin_staph/workflow2/quast/BCW_7425.1/quast.log

Main parameters: 
  Threads: 24, minimum contig length: 500, ambiguity: one, threshold for extensive misassembly size: 1000

Contigs:
  workflow2/megahit/BCW_7425.1/final.contigs.fa ==> final.contigs

2019-01-11 15:01:45
Running Basic statistics processor...
  Contig files: 
    final.contigs
  Calculating N50 and L50...
    final.contigs, N50 = 162146, L50 = 9, Total length = 4829247, GC % = 50.42, # N's per 100 kbp =  0.00
  Drawing Nx plot...
    saved to /group/weimergrp/smh/Dolphin_staph/workflow2/quast/BCW_7425.1/basic_stats/Nx_plot.pdf
  Drawing cumulative plot...
    saved

In [10]:
cat workflow2/quast/BCW_7425.1/report.tsv

Assembly	final.contigs
# contigs (>= 0 bp)	46
# contigs (>= 1000 bp)	43
# contigs (>= 5000 bp)	42
# contigs (>= 10000 bp)	41
# contigs (>= 25000 bp)	37
# contigs (>= 50000 bp)	28
Total length (>= 0 bp)	4830449
Total length (>= 1000 bp)	4829247
Total length (>= 5000 bp)	4824789
Total length (>= 10000 bp)	4818510
Total length (>= 25000 bp)	4757510
Total length (>= 50000 bp)	4414838
# contigs	43
Largest contig	406850
Total length	4829247
GC (%)	50.42
N50	162146
N75	106292
L50	9
L75	18
# N's per 100 kbp	0.00


> Assembly of __BCW_7425.1__ __*E. coli*__ shows **46 contigs**, total length of **4.83 MBp**, **50.42 % GC Content**, **N50 of 162 Kbp**

#### BCW_7425.2 Assembly Stats

> QUAST

In [11]:
quast -o workflow2/quast/BCW_7425.2 -t 24 workflow2/megahit/BCW_7425.2/final.contigs.fa

/share/apps/bio/bio/opt/quast-4.1/quast.py -o workflow2/quast/BCW_7425.2 -t 24 workflow2/megahit/BCW_7425.2/final.contigs.fa

Version: 4.1, build 26.05.2016 14:25

System information:
  OS: Linux-4.15.0-39-generic-x86_64-with-debian-buster-sid (linux_64)
  Python version: 2.7.15
  CPUs number: 24

Started: 2019-01-11 15:04:18

Logging to /group/weimergrp/smh/Dolphin_staph/workflow2/quast/BCW_7425.2/quast.log

Main parameters: 
  Threads: 24, minimum contig length: 500, ambiguity: one, threshold for extensive misassembly size: 1000

Contigs:
  workflow2/megahit/BCW_7425.2/final.contigs.fa ==> final.contigs

2019-01-11 15:04:19
Running Basic statistics processor...
  Contig files: 
    final.contigs
  Calculating N50 and L50...
    final.contigs, N50 = 143972, L50 = 7, Total length = 2828541, GC % = 32.61, # N's per 100 kbp =  0.00
  Drawing Nx plot...
    saved to /group/weimergrp/smh/Dolphin_staph/workflow2/quast/BCW_7425.2/basic_stats/Nx_plot.pdf
  Drawing cumulative plot...
    saved

In [12]:
cat workflow2/quast/BCW_7425.2/report.tsv

Assembly	final.contigs
# contigs (>= 0 bp)	42
# contigs (>= 1000 bp)	35
# contigs (>= 5000 bp)	33
# contigs (>= 10000 bp)	32
# contigs (>= 25000 bp)	25
# contigs (>= 50000 bp)	20
Total length (>= 0 bp)	2830691
Total length (>= 1000 bp)	2828039
Total length (>= 5000 bp)	2821762
Total length (>= 10000 bp)	2816197
Total length (>= 25000 bp)	2690262
Total length (>= 50000 bp)	2499182
# contigs	36
Largest contig	356733
Total length	2828541
GC (%)	32.61
N50	143972
N75	73507
L50	7
L75	14
# N's per 100 kbp	0.00


> Assembly of __BCW_7425.2__ __*Staphylococcus*__ shows **36 contigs**, total length of **2.83 MBp**, **32.61 % GC Content**, **N50 of 144 Kbp**